In [62]:
import numpy as np
from scipy.misc import derivative

class Neuron:
    def __init__(self, in_size, alpha):
        self.weights = np.random.uniform(low = -0.1, high = 0.1, size = in_size + 1)
        self.alpha = alpha
        self.out = 0
        
    def actvF(self, s):
        return 1/(1 + np.exp(-s))
        
    def spike(self, sample):
        s = sum(self.weights * np.append(sample, 1))
        self.out = self.actvF(s)
        return self.out
        
    def learn(self, sample, sigma, nxt_wt = None):
        sample = np.append(sample, 1)
        if nxt_wt is None:
            # this is a neuron of the last layer
            for i, e in enumerate(self.weights):
                self.weights[i] -= self.alpha * sigma * sample[i]
        else:
            for i, e in enumerate(self.weights):
                self.weights[i] -= self.alpha * sum(sigma * nxt_wt) * self.out * (1 - self.out) * sample[i]
            sigma = sum(sigma * nxt_wt) * derivative(self.actvF, sum(self.weights * sample))
        return sigma
            

In [63]:
class NLayer:
    def __init__(self, in_size, n_neurons, alpha, nxt = None):
        self.in_size = in_size
        self.n_neurons = n_neurons
        self.neurons = np.empty(n_neurons, dtype = object)
        for i in range(self.n_neurons):
            self.neurons[i] = Neuron(in_size, alpha)
        self.nxt = nxt
            
    def setInput(self, sample):
        self.in_signal = sample
        
    def process(self):
        self.output = np.empty(self.n_neurons)
        for i in range(self.n_neurons):
            self.output[i] = self.neurons[i].spike(self.in_signal)
            
    def learn(self, sigmas):
        if not self.nxt:
            # this is the last layer
            for k in range(self.n_neurons):
                self.neurons[k].learn(self.in_signal, sigmas[k])
            return sigmas
        else:
            self.sigmas = np.zeros(self.n_neurons)
            for k in range(self.n_neurons):
                nxt_wt = np.zeros(self.nxt.n_neurons)
                for i in range(self.nxt.n_neurons):
                    nxt_wt[i] = self.nxt.neurons[i].weights[k]
                self.sigmas[k] = self.neurons[k].learn(self.in_signal, sigmas, nxt_wt)
            return self.sigmas
                

In [64]:
class NNetwork:
    layers = None
    answer = None
    
    def __init__(self, in_size, out_size, hiddlist, alpha):
        self.layers = np.empty(len(hiddlist) + 1, dtype = object)
        self.layers[-1] = NLayer(hiddlist[-1], out_size, alpha)
        for i in range(1, len(hiddlist))[::-1]:
            self.layers[i] = NLayer(hiddlist[i-1], hiddlist[i], alpha, nxt = self.layers[i+1])
        self.layers[0] = NLayer(in_size, hiddlist[0], alpha, nxt = self.layers[1])
        
    def printLayers(self):
        for i, l in enumerate(self.layers):
            print(l.in_size, l.n_neurons, l.nxt)
            
    def process(self, sample):
        for i in range(len(self.layers)):
            if i == 0:
                self.layers[i].setInput(sample)
            else:
                self.layers[i].setInput(self.layers[i-1].output)
            self.layers[i].process()
            
        self.answer = self.layers[-1].output
        return self.answer
        
    def learn(self, d):
        sigmas = np.empty(len(d))
        for i in range(len(d)):
            sigmas[i] = (self.layers[-1].neurons[i].out - d[i]) * self.layers[-1].neurons[i].out * (1 - self.layers[-1].neurons[i].out)
            
        for i in range(len(self.layers))[::-1]:
            sigmas = self.layers[i].learn(sigmas)

In [65]:
np.random.seed(83838)
nnet = NNetwork(6, 2, [12], 0.1)
nnet.printLayers()

6 12 <__main__.NLayer object at 0x7f614dff0400>
12 2 None


In [66]:
#np.random.seed(83838)

for j in range(1000):
    x = np.random.uniform(low = -1, high = 1)
    if x >= 0:
        instance = np.array([0.9, 0.1, 0.1, 0.1, 0.1, 0.9])
        d = np.array([1., 0.])
    else:
        instance = np.array([0.1, 0.1, 0.9, 0.9, 0.1, 0.1])
        d = np.array([0., 1.])
    
    nnet.process(instance)
    nnet.learn(d)
    
instance = np.array([0.9, 0.1, 0.1, 0.1, 0.1, 0.9])
print(nnet.process(instance))

instance = np.array([0.1, 0.1, 0.9, 0.9, 0.1, 0.1])
print(nnet.process(instance))

instance = np.array([0.8, 0.15, 0.9, 0.19, 0.7, 0.95])
print(nnet.process(instance))

[ 0.7341879   0.26270394]
[ 0.28565547  0.71785276]
[ 0.61280433  0.3861741 ]
